# Emotion Detection con tecniche di Deep Learning

## Import delle librerie

In [ ]:
from tensorflow import keras
from keras.layers import Input, SimpleRNN, LSTM, Dense, Dropout, BatchNormalization, Flatten, Conv1D, Activation, MaxPool1D
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy

import os
import glob
import math
import wave #used to calculate audio file length
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Audio #used for cleaning too long cells output, used to play tracks
import librosa
import librosa.display

## Utility variables

In [ ]:
augmented_dataset = False #work with augmented dataset or with the default one

dataset_path = "Audio_Speech_Actors_01-24"
dataframe_path = "dataframe.csv"
dataframe_aug_path = "dataframe-aug.csv"

## Utility functions

In [ ]:
def plot_history(loss_values, accuracy_values, plot_file_name="plot"):
  fig, ax1 = plt.subplots(figsize=(10, 8))
  ax1.set_title("Accuracy and loss trend")
  
  #get number of cross-validation training iteration
  iteration_count = len(loss_values)
  #print x axis label
  ax1.set_xlabel('Epochs') 

  #print loss trend
  line1, = ax1.plot(range(1,iteration_count+1),loss_values,label='loss',color='orange')
  #show y axis label and color axis values
  ax1.set_ylabel('loss',color = line1.get_color())
  ax1.tick_params(axis='y', labelcolor=line1.get_color())
  #show legend
  _ = ax1.legend(loc='lower left')

  #clone y axis
  ax2 = ax1.twinx()
  #print accuracy trend
  line2, = ax2.plot(range(1,iteration_count+1),accuracy_values,label='accuracy')
  #show y axis label and color axis values
  ax2.set_ylabel('accuracy',color = line2.get_color())
  ax2.tick_params(axis='y', labelcolor=line2.get_color())
  #show legend
  _=ax2.legend(loc='upper right')

  fig.savefig("{}.png".format(plot_file_name))

## Dataset

Il dataset è una sottoporzione del dataset RAVDESS (Ryerson Audio-Visual Database of Emotional Speech and Song). Mentre il RAVDESS puro contiene 24.8GB di discorsi e di canzoni, audio e video, il RAVDESS Emotional speech audio contiene solo audio di discorsi.

Il dataset contiene 60 tracce audio per ognuno dei 24 attori, per un totale di 1440 file in formato war. Gli attori (12 maschi e 12 femmine) pronunciano 2 frasi (2 volte ciascuna) con un accento neutro del Norm America esprimendo emozioni differenti (calma, felicità, tristezza, rabbia, paura, sorpresa e disgusto) a 2 differenti livelli di intensità (medio, alto) più un'espressione neutra.

La **naming convention** dei file del dataset è la seguente: ognuno dei 1440 file è identificato da una successione di 7 identificatori numerici (es. 03-01-06-01-02-01-12.wav) e ognuno di questi numeri ha un significato specifico:

* **Modalità** (01: audio-video, 02: solo video, 03: solo audio)
  * in questa sottoporzione del dataset la modalità è sempre audio-only
* **Canale vocale** (01: parlato, 02: cantato)
  * in questa sottoporzione del dataset il canale vocale è sempre parlato 
* **Emozione** (01: neutra, 02: calma, 03: felicità, 04: tristezza, 05: rabbia, 06: paura, 07: disgusto, 08: sorpresa)
* **Intensità** (01: normale, 02: forte)
  * per l'emozione neutra non c'è intensità
* **Frase** (01: "Kids are talking by the door", 02: "Dogs are sitting by the door")
* **Ripetizione** (01: prima ripetizione, 02: seconda ripetizione)
* **Attore** (01...24, gli attori dispari sono maschi, mentre quelli pari sono femmine)

### Funzione di campionamento

Viene definita una utility function `sample_track` che restituisce in output il path di una traccia casuale campionato dal dataset.

In [ ]:
def sample_track():
  #choose a random folder
  folders = sorted(glob.glob(dataset_path+"/*"), key=len)
  random_folder = np.random.choice(folders)
  #choose and return a random track
  tracks = sorted(glob.glob(random_folder+"/*.wav"), key=len)
  random_track = np.random.choice(tracks)
  return random_track  

In [ ]:
#sampling function testing 
random_track = sample_track()
#track visualization
Audio(random_track)

### Caricamento dei file audio

La funzione [load](https://librosa.org/doc/main/generated/librosa.load.html) di librosa permette di caricare un file audio come una sequenza temporale di valori floating point.

* `path` indica alla funzione il path del file caricare

La funzione restituisce la sequenza di valori floating point e la frequenza di campionatura (espressa in Hz). Il valore di default per la frequenza di campionatura è di 22.050 Hz, il che significa che una traccia audio di 3 secondi viene caricata come un vettore di 66.150 valori floatin point.

In [ ]:
#load the audio track as a floating point time series
time_series, sampling_rate = librosa.load(path=random_track)
print("sampling_rate: {}".format(sampling_rate))
print("time_series length: {}".format(len(time_series)))

la funzione `get_audio_length` restituisce la durata in secondi della traccia audio passata in input. Usa il package [wave](https://docs.python.org/3/library/wave.html) per estrarre l'informazione dai file WAV.

* `path` è il percorso della traccia audio

La funzione viene usata per verificare la lunghezza della serie temporale estratta da librosa, basterà infatti moltiplicare la lunghezza del file per il rateo di campionatura e controllare se il valore coincide con il numero di valori estratti da librosa. 


In [ ]:
def get_audio_length(path):
    file = wave.open(path, 'r')
    frames = file.getnframes()
    #print("frames: ", frames)
    rate = file.getframerate()
    #print("frame rate: ", rate)
    return frames / float(rate)

In [ ]:
audio_length = get_audio_length(random_track)
print("audio length: {}".format(audio_length))
print("expected time series length: {}".format(math.ceil(audio_length * sampling_rate)))

In [ ]:
#display track spectrogram
spectrogram = librosa.feature.melspectrogram(y=time_series, sr=sampling_rate)
librosa.display.waveshow(y=spectrogram, sr=sampling_rate)

### Visualizzazione

Per fini di visualizzazione viene costruito un DataFrame Pandas ottenuto dalle informazioni codificate nei nomi dei singoli file. In particolare le colonne saranno le seguenti: `Path`, `Emotion`, `Intensity`, `Statement`, `Actor`, `Gender`.

Prima vengono generati array di valori in base ai codici contenuti nei nomi dei file, poi quegli array vengono convertiti in dataframe con una singola colonna e infine i dataframes vengono concatenati per comporre il dataframe obiettivo.

In [ ]:
directories = os.listdir(dataset_path)

#support arrays to store database values
emotion_col = []; intensity_col = []; statement_col = []; actor_col = []; gender_col = []; path_col = []

#iterate over dataset folders (every folder is relative to a different actor)
for dir_name in directories:
  dir_files = os.listdir(dataset_path + "/" + dir_name)
  #iterate over files in the current directory
  for file_name in dir_files:
    #file name splitting: first remove WAV extension then split over '-' chars
    file_parts = file_name.split('.')[0].split('-')
    #store each value into the relative array
    emotion_col.append(int(file_parts[2]))
    intensity_col.append(int(file_parts[3]))
    statement_col.append(int(file_parts[4]))
    actor_col.append(int(file_parts[6]))
    gender_col.append(int(file_parts[6])%2) # %2 to keep only odd/even information
    path_col.append(dataset_path + "/" + dir_name + "/" + file_name)

#convert arrays to dataframes and chain them in a single dataframe
emotion_df = pd.DataFrame(emotion_col, columns=['Emotion'])
intensity_df = pd.DataFrame(intensity_col, columns=['Intensity'])
statement_df = pd.DataFrame(statement_col, columns=['Statement'])
actor_df = pd.DataFrame(actor_col, columns=['Actor'])
gender_df = pd.DataFrame(gender_col, columns=['Gender'])
path_df = pd.DataFrame(path_col, columns=['Path'])

#convert integer in dataframe with their value following the naming convention
#  inplace param added to edit the current dataframe instead of creating a new one
emotion_df.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fearful', 7:'disgust', 8:'surprised'}, inplace=True)
intensity_df.replace({1:'normal', 2:'strong'}, inplace=True)
statement_df.replace({1:'Kids are talking by the door', 2:'Dogs are sitting by the door'}, inplace=True)
gender_df.replace({1:'male', 0:'female'}, inplace=True)

files_df = pd.concat([path_df, emotion_df, intensity_df, statement_df, actor_df, gender_df], axis=1) #axis=1 to concatenate columns instead of rows
files_df

Vengono contati i valori assumibili dalla colonna Emotions del dataframe per definire il numero di classi possibili

In [ ]:
classes_count = files_df["Emotion"].nunique()
print("Number of possible classes (emotions): {}".format(classes_count))

### Augmentation

Sono state usate tecniche di data augmentation per incrementare la dimensione del dataset. Per la fonte dei metodi si rimanda al notebook [Audio Emotion | Data Augmentation](https://www.kaggle.com/code/ejlok1/audio-emotion-part-5-data-augmentation).

Le tecniche usate sono le seguenti:

* static noise
* shift
* stretch
* pitch
* dynamic change
* speed and pitch

Nei sottoparagrafi successivi vengono implementati i metodi e testati su un file audio di test.

In [ ]:
time_series, sampling_rate = librosa.load(path=random_track)
Audio(time_series, rate=sampling_rate)

#### Static noise

Il metodo `add_static_noise` aggiunge del rumore di fondo al file passato in input. 

In [ ]:
def add_static_noise(time_series):
    noise_amp = 0.05 * np.random.uniform() * np.amax(time_series)
    return time_series.astype('float64') + noise_amp * np.random.normal(size=time_series.shape[0])

In [ ]:
print(len(time_series))
noised_time_series = add_static_noise(time_series)
print(len(noised_time_series))
Audio(noised_time_series, rate=sampling_rate)

#### Shift

In [ ]:
def shift_audio(time_series):
    shift_range = int(np.random.uniform(low=5,high=5) * 1000)
    return np.roll(time_series,shift_range)

In [ ]:
shifted_time_series = shift_audio(time_series)
print(len(shifted_time_series))
Audio(shifted_time_series, rate=sampling_rate)

#### Stretch

In [ ]:
def stretch_audio(time_series):
    return librosa.effects.time_stretch(time_series,rate=0.9)

In [ ]:
stretched_time_series = stretch_audio(time_series)
Audio(stretched_time_series, rate=sampling_rate)

#### Pitch

https://librosa.org/doc/main/generated/librosa.effects.pitch_shift.html

In [ ]:
def pitch_audio(time_series, sampling_rate):
    bins_per_octave = 12
    pitch_pm = 2
    pitch_change = pitch_pm * 2 * (np.random.uniform())
    return librosa.effects.pitch_shift(y=time_series.astype('float64'),
                                       sr=sampling_rate,
                                       n_steps=pitch_change,
                                       bins_per_octave=bins_per_octave)

In [ ]:
pitched_time_series = pitch_audio(time_series, sampling_rate)
Audio(pitched_time_series, rate=sampling_rate)

#### Dynamic change

In [ ]:
def dynamic_change(time_series):
    return time_series * np.random.uniform(low=-0.5,high=7)

In [ ]:
changed_time_series = dynamic_change(time_series)
Audio(changed_time_series, rate=sampling_rate)

#### Speed and pitch

In [ ]:
def speed_and_pitch(time_series):
    length_change = np.random.uniform(low=0.8,high=1)
    speed_factor = 1.2 / length_change
    tmp = np.interp(np.arange(0,len(time_series),speed_factor), np.arange(0,len(time_series)), time_series)
    minlen = min(time_series.shape[0], tmp.shape[0])
    time_series *= 0
    time_series[0:minlen] = tmp[0:minlen]
    return time_series

In [ ]:
sap_time_series = speed_and_pitch(time_series)
Audio(sap_time_series, rate=sampling_rate)

### Features

Dal dataset vengono estratte le features usando le funzioni offerte dalla libreria librosa:

* [chroma](https://librosa.org/doc/main/generated/librosa.feature.chroma_stft.html): **TODO** (12 valori per ogni file)
* [mel spctrogram](https://librosa.org/doc/main/generated/librosa.feature.melspectrogram.html): spettrogramma dove le frequenze sono state convertine nella scala MEL (128 valori per ogni file)
* [mfcc](https://librosa.org/doc/main/generated/librosa.feature.mfcc.html): **TODO** (40 valori per ogni file)
* [spectral contrast](https://librosa.org/doc/main/generated/librosa.feature.spectral_contrast.html): **TODO** (7 valori per ogni file)
* [tonnetz](https://librosa.org/doc/main/generated/librosa.feature.tonnetz.html): **TODO** (6 valori per ogni file)

I valori indicati tra parentesi (12,128,40,7,6 = **193 features**) indicano il numero di colonne di ogni feature, che verranno aggiunte al dataframe Pandas, corredato con l'informazione di classe (corrispondente all'emozione espressa nella traccia audio) del file corrispondente.

<!-- #### Short-Time Fourier Transform -->

<!--
La **Fourier Transform** (FT) è una funzione matematica in grado di decomporre un segnale nelle sue frequenze costituenti corredate dalle relative magnitudo. La FT permette di passare dal dominio del tempo (time-domain) a quello della frequenza (frequency-domain).

<div>
<img src="https://drive.google.com/uc?id=12vRYA33koEKGD2fvZOYKUxMBhkXzd3ab"/>
</div>

La libreria librosa contiene una funzione per il calcolo della **Discrete Fourier Transform** (DFT), chiamata anche **Fast Fourier Transform** (FFT), funzione matematica che si differenzia dalla FT per il fatto di prendere un input discreto, piuttosto che un input continuo come nella FT standard.

La frequenza cambia nel tempo, quindi una FT su tutta la traccia audio può non essere rappresentativa. La **Short-Time Fourier Transform** (STFT) effettua una FT su sottoporzioni della traccia stessa.

La STFT viene calcolata usando la funzione [stft](https://librosa.org/doc/main/generated/librosa.stft.html) di librosa, che di norma restituisce un vettore di numeri complessi (float64), che codificano fase e ampiezza del segnale audio, ma la fase, oltre a non venire percepita dall'essere umano, non è utile ai fini dell'esperimento, quindi convertono i valori della sequenza temporale in valori assoluti (float32).

**TODO** parlare del binning
-->

#### Estrazione

La funzione `get_features` estrae le 193 features dal file passato in input e restituisce 5 array, uno per ogni categoria di feature sopraelencata.

* `file_path` è il percorso del file per il quale estrarre le features

In [ ]:
def get_features(time_series, sampling_rate=22050):
  stft = np.abs(librosa.stft(time_series))

  chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sampling_rate).T,axis=0)
  mel_spectrogram = np.mean(librosa.feature.melspectrogram(y=time_series, sr=sampling_rate).T,axis=0)
  mfcc = np.mean(librosa.feature.mfcc(y=time_series, sr=sampling_rate, n_mfcc=40).T,axis=0)
  spectral_contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sampling_rate).T,axis=0)
  tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(time_series), sr=sampling_rate).T,axis=0)
  return chroma, mel_spectrogram, mfcc, spectral_contrast, tonnetz

La funzione `get_features` viene usata per definire quante feature verranno passate in input ai vari modelli.

In [ ]:
features_count = sum(len(f) for f in get_features(time_series))
#features_count = len(get_features(random_track))

print("features for every file: {}".format(features_count))

La funzione `get_feature_dataframe` restituisce un dataframe Pandas contenente le feature estratte da ogni file del dataset. Il dataframe conterrà quindi una riga per ogni file nel dataset (**1440 righe** nella forma standard, **10.080 righe** nella versione estesa applicando la data augmentation) e una colonna per ogni feature più una colonna per ogni classe possibile (**193 + 8 = 201 colonne**).

Per la classe di appartenenza è stata utilizzata una rappresentazione in One Hot Encode (usando la funzione [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) di Pandas) per evitare che il modello apprenda relazioni in realtà non esistenti tra i valori della classe dei dati.

In [ ]:
def get_time_series_list(augment_dataset):
  #define time series lists (for standard ts and for augmented ts)
  ts_list = []; ts_static_noise = []; ts_shift = []; ts_stretch = []; ts_pitch = []; ts_dyn_change = []; ts_sep = []
  #time series extraction from files
  for _,row in files_df.iterrows():
    time_series,_ = librosa.load(path=row['Path'])
    ts_list.append(time_series)    
    #populate augmented time series lists  
    if augment_dataset:
      ts_static_noise.append(add_static_noise(time_series))
      ts_shift.append(add_static_noise(time_series))
      ts_stretch.append(add_static_noise(time_series))
      ts_pitch.append(add_static_noise(time_series))
      ts_dyn_change.append(add_static_noise(time_series))
      ts_sep.append(add_static_noise(time_series))
  #merge time series lists
  if augment_dataset:
    ts_list = np.concatenate([ts_list, ts_static_noise, ts_shift, ts_stretch, ts_pitch, ts_dyn_change, ts_sep])
  return ts_list

In [ ]:
def get_feature_dataframe(augment_dataset=False, standardize=True):
  #support arrays to store feature values
  chroma_col = []; mel_spectrogram_col = []; mfcc_col = []; spectral_contrast_col = []; tonnetz_col = []

  #features extraction from time series
  for time_series in get_time_series_list(augment_dataset):
    chroma, mel_spectrogram, mfcc, spectral_contrast, tonnetz = get_features(time_series)

    chroma_col.append(chroma)
    mel_spectrogram_col.append(mel_spectrogram)  
    mfcc_col.append(mfcc)
    spectral_contrast_col.append(spectral_contrast)
    tonnetz_col.append(tonnetz)

  #convert feature arrays to pandas dataframes and chain them them to obtain the feature dataframe
  chroma_df = pd.DataFrame(chroma_col)
  chroma_df.rename(columns=lambda i: 'Chroma_' + str(i+1), inplace=True)

  mel_df = pd.DataFrame(mel_spectrogram_col)
  mel_df.rename(columns=lambda i: 'MelSpectrogram_' + str(i+1), inplace=True)

  mfcc_df = pd.DataFrame(mfcc_col)
  mfcc_df.rename(columns=lambda i: 'MFCC_' + str(i+1), inplace=True)

  contrast_df = pd.DataFrame(spectral_contrast_col)
  contrast_df.rename(columns=lambda i: 'SpectralContrast_' + str(i+1), inplace=True)

  tonnetz_df = pd.DataFrame(tonnetz_col)
  tonnetz_df.rename(columns=lambda i: 'Tonnetz_' + str(i+1), inplace=True)

  features_df = pd.concat([chroma_df, mel_df, mfcc_df, contrast_df, tonnetz_df], axis=1)

  #standardize features
  if standardize:
    for column in features_df:
      features_df[column] = (features_df[column] - features_df[column].mean()) / features_df[column].std()

  emotion_df_ohe = pd.get_dummies(data=emotion_df, prefix="emotion", columns=["Emotion"])

  if(augment_dataset):
    actor_df2 = pd.concat([actor_df]*7, ignore_index=True)
    emotion_df_ohe2 = pd.concat([emotion_df_ohe]*7, ignore_index=True)
    return pd.concat([actor_df2, features_df, emotion_df_ohe2], axis=1)
  else:
    return pd.concat([actor_df, features_df, emotion_df_ohe], axis=1)

Per valutare l'efficacia dell'augmentation del dataset sono stati condotti due training usando lo stesso modello (LSTM), il primo utilizzando il dataset standard e il secondo usando la versione estesa.

### Creazione del dataframe

Il seguente blocco di codice permette di decidere se ricreare il dataframe o se caricare la versione salvata su Drive, risparmiando il tempo necessario per la costruzione.

In [ ]:
def get_dataframe(path):
  if os.path.exists(path):
    #create dataframe from CSV file
    dataframe = pd.read_csv(path)
  else:
    #create dataframe from dataset
    print("creating dataframe...")
    dataframe = get_feature_dataframe(augment_dataset=augmented_dataset)
    print("dataframe creation completed!")
    #save dataframe ad CSV file
    dataframe.to_csv(path, index=False) #index param avoid to save row names (unused information)
  return dataframe

In [ ]:
dataframe = get_dataframe(dataframe_path)
dataframe_aug = get_dataframe(dataframe_aug_path)
print("standard dataframe shape: {}, augmented dataframe shape: {}".format(dataframe.shape, dataframe_aug.shape))

Viene usata la funzione [info](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html) dei DataFrame Pandas per visualizzare informazioni interessanti sul DataFrame. A causa del elevato numero di colonne è necessario impostare il parametro `verbose` per forzare la visualizzazione completa.

In [ ]:
dataframe.info()
#dataframe.info(verbose=True) #used only for debug purpose

In [ ]:
dataframe.describe().transpose()

### Splitting

In [ ]:
available_actor_indexes = dataframe['Actor'].unique()
test_actor_indexes = [1, 12, 7, 24]
validation_actor_indexes = [2,13,8,23]
#subtraction with list comprehension
training_actor_indexes = [x for x in available_actor_indexes if x not in test_actor_indexes and x not in validation_actor_indexes]
print("training actors are {} over a total of {}"
      .format(len(training_actor_indexes),len(available_actor_indexes)))

#test and validation set are sampled from not-augmented dataset
test_set = dataframe.loc[dataframe['Actor'].isin(test_actor_indexes)]
validation_set = dataframe.loc[dataframe['Actor'].isin(validation_actor_indexes)]

#project validation set to get separately data and classes
validation_data = validation_set.iloc[:,1:features_count+1]
validation_classes = validation_set.iloc[:,features_count+1:]
#reshape data
validation_data = validation_data.to_numpy()
validation_data = validation_data.reshape((validation_data.shape[0], validation_data.shape[1], 1))

#display dataset subset shape
print("Shapes -> validation set: {}, test set: {}"
.format(validation_set.shape, test_set.shape))

## Funzione di training

In [ ]:
def cross_validation(model, iterations=1):
  #define array of metrics, one value for every cross-validation iteration
  model_loss = []; model_accuracy = []

  #on every iteration fix a different actor, train on the other actors and test on him
  for iteration in range(iterations):
    fold_index = 0 #used to print the index of the current processing fold

    for test_actor_index in training_actor_indexes:
      
      cross_train_actor_indexes = [x for x in training_actor_indexes if x != test_actor_index]
      #define train and test set for the current iteration (training set created from augmented dataframe)
      cross_test_set = dataframe.loc[dataframe['Actor'].isin([test_actor_index])]
      cross_training_set = dataframe_aug.loc[dataframe_aug['Actor'].isin(cross_train_actor_indexes)]      
      #project training set and test set to get separately data and classes
      train_data = cross_training_set.iloc[:,1:features_count+1]
      train_classes = cross_training_set.iloc[:,features_count+1:]
      test_data = cross_test_set.iloc[:,1:features_count+1]
      test_classes = cross_test_set.iloc[:,features_count+1:]
      #reshape data
      train_data = train_data.to_numpy()
      train_data = train_data.reshape((train_data.shape[0], train_data.shape[1], 1))
      test_data = test_data.to_numpy()
      test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], 1))
      # fit the classifier
      fold_index += 1
      history = model.fit(x=train_data, y=train_classes, verbose=0)
      # compute the score and record it
      test_result = model.evaluate(x=test_data, y=test_classes, verbose=0)
      model_loss.append(test_result[0]) #metrics are loss and accuracy
      model_accuracy.append(test_result[1])

      print("Iter {}/{}, Fold {}/{}, {} data, fit loss: {:.4f}, fit accuracy: {:.4f}, test loss: {:.4f}, test accuracy {:.4f}".format(
          iteration+1, iterations, fold_index, len(training_actor_indexes), train_data.shape[0],
          history.history['loss'][0], history.history['accuracy'][0], test_result[0], test_result[1]))

  return model_loss, model_accuracy


## RNN base

Il primo esperiemnto viene condotto usando una RNN many-to-one standard, cioè con un unico nodo. 



![alt text](https://biolab.csr.unibo.it/ferrara/Courses/DL/Tutorials/RNN/ManyToOne_RNN.png)

### Definizione del modello

[SimpleRNN](https://keras.io/api/layers/recurrent_layers/simple_rnn/)

In [ ]:
def build_simple_rnn(n_feature,n_classes):
  model = keras.Sequential(
      [
        Input(shape=(n_feature,1)),
        #SimpleRNN(n_classes, activation="softmax")
        
        SimpleRNN(8, activation="relu"),
        Dense(n_classes, activation="softmax")
      ]
    )

  return model

### Creazione del modello

In [ ]:
simple_rnn = build_simple_rnn(features_count, classes_count)

### Visualizzazione del modello

In [ ]:
simple_rnn.summary()

In [ ]:
plot_model(simple_rnn, show_shapes=True, show_layer_names=False)

### Compilazione del modello

[Adam](https://keras.io/api/optimizers/adam/)

In [ ]:
loss = BinaryCrossentropy()
optimizer = Adam()

simple_rnn.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

### Training del modello

In [ ]:
#loss, accuracy = cross_validation(simple_rnn,3)

In [ ]:
#plot_history(loss, accuracy)

## LSTM

### Definizione del modello

In [ ]:
def build_lstm(n_feature,n_classes):
  model = keras.Sequential(
      [
        Input(shape=(n_feature,1)),
        LSTM(8, activation="relu"),
        Dense(n_classes, activation="softmax")
      ]
    )

  return model

### Creazione del modello

In [ ]:
lstm = build_lstm(features_count, classes_count)

### Visualizzazione del modello

In [ ]:
lstm.summary()

In [ ]:
plot_model(lstm, show_shapes=True, show_layer_names=False)

### Compilazione del modello

In [ ]:
loss = BinaryCrossentropy()
optimizer = Adam()

lstm.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

### Training del modello

In [ ]:
#loss, accuracy = cross_validation(lstm)

In [ ]:
#plot_history(loss, accuracy)

## Advanced Net

### Definizione del modello

In [ ]:
def build_model(n_feature,n_classes):
  model = keras.Sequential(
      [
        Input(shape=(n_feature,1)),
        Conv1D(32, 8, padding="same"),
        BatchNormalization(axis=-1),
        Activation("relu"),
        Dropout(0.3),
        MaxPool1D(),
        Conv1D(64, 6, padding="same"),
        BatchNormalization(axis=-1),
        Activation("relu"),
        Dropout(0.3),
        MaxPool1D(),
        Conv1D(128, 4, padding="same"),
        BatchNormalization(axis=-1),
        Activation("relu"),
        Dropout(0.3),
        MaxPool1D(),  
        Flatten(),
        Dense(128),
        BatchNormalization(axis=-1),
        Activation("relu"),
        Dense(n_classes, activation="softmax")
      ]
    )
  return model

### Creazione del modello

In [ ]:
model = build_model(features_count, classes_count)

### Visualizzazione del modello

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=False)

### Compilazione del modello

In [ ]:
loss = BinaryCrossentropy()
optimizer = Adam(learning_rate=0.0001)

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

### Training del modello

In [ ]:
loss, accuracy = cross_validation(model,20)

In [ ]:
plot_history(loss, accuracy)

## Risultati del training

In [ ]:
#simple_rnn.evaluate(validation_data, validation_classes)
#lstm.evaluate(validation_data, validation_classes)
model.evaluate(validation_data, validation_classes)

In [ ]:
#LISTA DI RISULTATI
# acc: 0.3292   3x lstm, neuroni: (32,32,32) , 10x CV
# acc: 0.3542   1x lstm, neuroni
# acc: 0.3333   3x lstm, neuroni (8,8,8), 10x CV